<a href="https://colab.research.google.com/github/DanZter/BERT_framework/blob/master/BERT_on_steroids_X_multicore_tpu_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [2]:
pip install torch===1.5.1 torchvision===0.6.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 753.2MB 23kB/s 
     |████████████████████████████████| 6.6MB 50.8MB/s 
  Found existing installation: torch 1.5.1+cu101
    Uninstalling torch-1.5.1+cu101:
      Successfully uninstalled torch-1.5.1+cu101
  Found existing installation: torchvision 0.6.1+cu101
    Uninstalling torchvision-0.6.1+cu101:
      Successfully uninstalled torchvision-0.6.1+cu101


In [3]:
VERSION = "1.5"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4139  100  4139    0     0  67852      0 --:--:-- --:--:-- --:--:-- 67852
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-1.5 ...
Uninstalling torch-1.5.1:
  Successfully uninstalled torch-1.5.1
     |████████████████████████████████| 61kB 2.1MB/s 
Uninstalling torchvision-0.6.1:
  Successfully uninstalled torchvision-0.6.1
ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Trace

In [4]:
# imports pytorch
import torch

# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.distributed.parallel_loader as pl

In [5]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.2.0
Running on TPU  ['10.15.135.82:8470']
INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [6]:
!pip install transformers

     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 1.1MB 15.8MB/s 
     |████████████████████████████████| 890kB 22.9MB/s 
     |████████████████████████████████| 3.0MB 32.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=f1352f6121064c447dc54f8ce5565ef81f603eca857254e0996a3bea0128184e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [7]:
# pip uninstall torchvision-nightly-cp36-cp36m-linux_x86_64.whl -y

In [8]:
import pandas as pd
import numpy as np
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
import torch.nn as nn
from sklearn import model_selection
from scipy import stats

import warnings
warnings.filterwarnings("ignore")
# import logging
# logging.basicConfig(level=logging.ERROR)

In [13]:
class BERTBaseUncased(nn.Module):
    def __init__(self, bert_path):
        super(BERTBaseUncased, self).__init__()
        self.bert_path = bert_path
        self.bert = transformers.BertModel.from_pretrained(self.bert_path)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, 30)       # pool output has 768 features, 30 targets

    def forward(self, ids, mask, token_type_ids):
        _, o2 = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)  # not using o1 - Sequential Output
        bo = self.bert_drop(o2)                                                     # using o2 - Pooled Output
        return self.out(bo)

class BERTDatasetTraining:
    def __init__(self, qtitle, qbody, answer, targets, tokenizer, max_len):
        self.qtitle = qtitle
        self.qbody = qbody
        self.answer = answer
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.targets = targets                   # numpy array of size: no. of samples * 30(targets)

    def __len__(self):
        return len(self.answer)                  # returns length of title or body or answer

    def __getitem__(self, item):                 # takes in index and returns output
        question_title = str(self.qtitle[item])
        question_body = str(self.qbody[item])
        answer = str(self.answer[item])

        # [CLS] [Q-TITLE] [Q-BODY] [SEP] [ANSWER] [SEP]

        inputs = self.tokenizer.encode_plus(
            question_title + " " + question_body,
            answer,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True 
        )

        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]

        padding_len = self.max_len - len(ids)
        ids = ids + ([0] * padding_len)
        token_type_ids = token_type_ids + ([0] * padding_len)
        mask = mask + ([0] * padding_len)

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "targets": torch.tensor(self.targets[item, :], dtype=torch.long)
        }

def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs, targets)

def train_loop_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    for bi, d in enumerate(data_loader):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        loss = loss_fn(outputs, targets)
        loss.backward()
        # optimizer.step()                                         # cuda
        xm.optimizer_step(optimizer)                              # tpu
        if scheduler is not None:
            scheduler.step()
        if bi % 10 == 0:
            print(f"bi={bi}, loss={loss}")

def eval_loop_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    for bi, d in enumerate(data_loader):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]
        targets = d["targets"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        loss = loss_fn(outputs, targets)

        fin_targets.append(targets.cpu().detach().numpy())
        fin_outputs.append(outputs.cpu().detach().numpy())        # Linear Layer: can apply sigmoid here

        return np.vstack(fin_outputs), np.vstack(fin_targets)

def run(index, flags):

    # MAX_LEN = 512
    # TRAIN_BATCH_SIZE = 4
    # EPOCHS = 20

    flags['TRAIN_BATCH_SIZE'] = 4
    flags['TEST_BATCH_SIZE'] = 4
    flags['MAX_LEN'] = 512
    flags['EPOCHS'] = 20
    flags['seed'] = 1234  
    torch.manual_seed(flags['seed'])

    dfx = pd.read_csv("/content/drive/My Drive/Colab Notebooks/input/google_quest_train.csv").fillna("none")
    df_train, df_valid = model_selection.train_test_split(dfx, random_state = 42, test_size = 0.1)
    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)

    sample = pd.read_csv("/content/drive/My Drive/Colab Notebooks/input/google_quest_sample_submission.csv")
    target_cols = list(sample.drop("qa_id", axis=1).columns)
    train_targets = df_train[target_cols].values
    valid_targets = df_valid[target_cols].values

    tokenizer =transformers.BertTokenizer.from_pretrained("/content/drive/My Drive/Colab Notebooks/input/bert_base_uncased")

    train_dataset = BERTDatasetTraining(
        qtitle=df_train.question_title.values,
        qbody=df_train.question_body.values,
        answer=df_train.answer.values,
        targets=train_targets,
        tokenizer=tokenizer,
        max_len=flags['MAX_LEN']
    )
    train_sampler = torch.utils.data.DistributedSampler(
       train_dataset,
       num_replicas=xm.xrt_world_size(),
       rank=xm.get_ordinal(),
       shuffle=True 
    )
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size= flags['TRAIN_BATCH_SIZE'],
        sampler=train_sampler
    )
    ############
    valid_dataset = BERTDatasetTraining(
        qtitle=df_valid.question_title.values,
        qbody=df_valid.question_body.values,
        answer=df_valid.answer.values,
        targets=valid_targets,
        tokenizer=tokenizer,
        max_len=flags['MAX_LEN']
    )
    valid_sampler = torch.utils.data.DistributedSampler(
       valid_dataset,
       num_replicas=xm.xrt_world_size(),
       rank=xm.get_ordinal()
    )
    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=flags['TEST_BATCH_SIZE'],
        sampler=valid_sampler
    )

    # device = "cuda"                   # cuda
    device = xm.xla_device()            # tpu
    lr = 3e-5 *xm.xrt_world_size()
    num_train_steps = int(len(train_dataset)/ flags['TRAIN_BATCH_SIZE'] / xm.xrt_world_size() * flags['EPOCHS'])
    model = BERTBaseUncased("/content/drive/My Drive/Colab Notebooks/input/bert_base_uncased").to(device)

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=num_train_steps
    )
    for epoch in range(flags['EPOCHS']):
      para_loader = pl.ParallelLoader(train_data_loader, [device])
      train_loop_fn(para_loader.per_device_loader(device), model, optimizer, device, scheduler)

      para_loader = pl.ParallelLoader(valid_data_loader, [device])
      o, t = eval_loop_fn(para_loader.per_device_loader(device), model, device)

      spear = []
      for jj in range(t.shape[1]):
          p1 = list(t[:,jj])
          p2 = list(o[:, jj])
          coef, _ = np.nan_to_num(stats.spearmanr(p1,p2))
          spear.append(coef)
      spear = np.mean(spear)
      print(f"epoch = {epoch}, spearman = {spear}")
      # torch.save(model.state_dict(), "model.bin")         # cuda
      # xm.save(model.state_dict(), "model.bin")          # tpu

# if __name__=="__main__":
#   xmp.spawn(run, nprocs=1)

# # Spawns eight of the map functions, one for each of the eight cores on
# # the Cloud TPU
# flags = {}
# # Note: Colab only supports start_method='fork'
# xmp.spawn(run, args=(flags,), nprocs=1, start_method='fork')

# # xmp.spawn(run, args=(), start_method='fork', nprocs=1)
flags = {}
xmp.spawn(run, args=(flags,), nprocs=8, start_method='fork')










bi=0, loss=0.7525567412376404
bi=0, loss=0.7461388111114502
bi=0, loss=0.8078871965408325
bi=0, loss=0.7887917757034302
bi=0, loss=0.7751476168632507
bi=0, loss=0.752004086971283
bi=0, loss=0.7817627787590027
bi=0, loss=0.7815744280815125
bi=10, loss=0.5041784644126892
bi=10, loss=0.3669978678226471
bi=10, loss=0.3658598065376282
bi=10, loss=0.45700758695602417
bi=10, loss=0.4400368332862854
bi=10, loss=0.40068110823631287
bi=10, loss=0.4434146285057068
bi=10, loss=0.3725464940071106
bi=20, loss=0.3287964165210724
bi=20, loss=0.41121095418930054
bi=20, loss=0.28016239404678345
bi=20, loss=0.33210667967796326
bi=20, loss=0.38915687799453735
bi=20, loss=0.3130435049533844
bi=20, loss=0.4513082504272461
bi=20, loss=0.4878334403038025
bi=30, loss=0.4104171395301819
bi=30, loss=0.33002346754074097
bi=30, loss=0.3433081805706024
bi=30, loss=0.3132987320423126
bi=30, loss=0.41064614057540894
bi=30, loss=0.5645810961723328
bi=30, loss=0.2972920536994934
bi=30, loss=0.3348168730735779
bi=40, lo